# Apache Airflow для аналитика: Проектная работа №0

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
# Импорт необходимых библиотек
import pandas as pd
import datetime as dt
import sqlite3

In [2]:
# Создадим базу данных с которой будем работать
CON = sqlite3.connect('sqlite3.db')

In [13]:
# Напишите функцию которая скачивает данные с гитхаба
# Функция принимает на вход дату
# Функция должна вернуть список словарей или датафрейм, можно(лучше) использовать можно использоват pandas.read_csv()

# С помощью этой функции вы считаете данные по 2 ссылкам
# ссылка на гитхаб с курсами валют - https://github.com/datanlnja/airflow_course/tree/main/excangerate
# ссылка на гитхаб с данными о продажах - https://github.com/datanlnja/airflow_course/tree/main/data

# сырые данные о курсах валют - пример - 
# https://raw.githubusercontent.com/datanlnja/airflow_course/main/excangerate/2021-01-01.csv
# сырые данные с данными о продажах - пример - 
# https://raw.githubusercontent.com/datanlnja/airflow_course/main/data/2021-01-01.csv

def extract_data(url, date):
  final_url = url+ date+ '.csv'
  return pd.read_csv(final_url)

In [4]:
# Напишите функцию которая принимает на вход список словарей, или пандас датафрейм
# и загружает в табличку в sqlite, можно использоват pandas.to_sql()

def insert_to_db(data, table_name, conn):
  #data["insert_time"] = pd.to_datetime("now")
  data.to_sql(table_name, conn, if_exists='append', index=False)

In [5]:
# Напишите функцию которая объединит данные по ключу или паре ключей
# На выходе возвращает данные, рекомендую использовать pandas.DataFrame

def merge_data(data1, data2):
  return data1.merge(data2, on='date')

In [7]:
# Запустите ваш код в функции main

# Напишите генерацию дат, так чтобы у вас получился список
# 2021-01-02, 2021-01-03 ... etc
# Нужны даты с 2021-01-01 по 2021-01-04

start_date = dt.datetime(2021, 1, 1)
end_date = dt.datetime(2021, 1, 4)
dates_list = pd.date_range(
    min(start_date, end_date),
    max(start_date, end_date)
).strftime('%Y-%m-%d').tolist()

def main(date, conn):

  # Выгружаем данные по валютам и из источника
  currency = extract_data('https://raw.githubusercontent.com/datanlnja/airflow_course/main/excangerate/', date)
  data = extract_data('https://raw.githubusercontent.com/datanlnja/airflow_course/main/data/', date)

  # Объедините данные в 1 таблицу
  mg_data = merge_data(currency, data)
  print(mg_data)
  # Вставляем данные в БД в таблицу data
  insert_to_db(mg_data, 'my_data', conn=CON)

# Пройдемся по списку дат и выполним скрипт
for date in dates_list:
  main(date, CON)

         date currency_from currency_to  amount currency  value
0  2021-01-01           eur         usd    1.21      EUR     38
1  2021-01-01           eur         usd    1.21      EUR     65
2  2021-01-01           eur         usd    1.21      EUR     74
3  2021-01-01           eur         usd    1.21      EUR     42
4  2021-01-01           eur         usd    1.21      EUR     23
5  2021-01-01           eur         usd    1.21      EUR     48
6  2021-01-01           eur         usd    1.21      EUR     86
7  2021-01-01           eur         usd    1.21      EUR     74
8  2021-01-01           eur         usd    1.21      EUR     24
         date currency_from currency_to  amount currency  value
0  2021-01-02           eur         usd    1.22      EUR     27
1  2021-01-02           eur         usd    1.22      EUR     71
2  2021-01-02           eur         usd    1.22      EUR     39
3  2021-01-02           eur         usd    1.22      EUR     94
4  2021-01-02           eur         usd 

In [12]:
# чтобы проверить решение можете обратиться к вашей базе данных таким образом

import sqlite3

# Создаем подключение к базе данных
conn = sqlite3.connect('sqlite3.db')

# Создаем курсор для выполнения SQL-запросов
cursor = conn.cursor()

# Выполняем запрос к таблице
cursor.execute("SELECT * FROM my_data")

# Извлекаем все строки из результата запроса
rows = cursor.fetchall()

# Выводим результаты
for row in rows:
    print(row)

# Выполняем запрос к таблице
cursor.execute("SELECT sum(amount*value) FROM my_data")

# Извлекаем все строки из результата запроса
row = cursor.fetchall()

# Выводим результат
print(row)
    
# Закрываем соединение
conn.close()

('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 38)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 65)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 74)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 42)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 23)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 48)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 86)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 74)
('2021-01-01', 'eur', 'usd', 1.21, 'EUR', 24)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 27)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 71)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 39)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 94)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 45)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 3)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 49)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 96)
('2021-01-02', 'eur', 'usd', 1.22, 'EUR', 36)
('2021-01-03', 'eur', 'usd', 1.22, 'EUR', 100)
('2021-01-03', 'eur', 'usd', 1.22, 'EUR', 93)
('2021-01-03', 'eur', 'usd', 1.22, 'EUR', 56)
('2021-01-03', 'eur', 'usd', 1.22,

Ответ: 2229.08


Ссылка на Google Collab: https://colab.research.google.com/drive/1qPknxS-lxj6Z_W6_iK6jp08RVWltagAs#scrollTo=oKOYD6FjsRGt


_____________________________________________________________________________________________________________________________
Проектную работу выполнила: аналитик данных Малахова Наталья

Мой телеграм-канал: [Дневник аналитика](https://t.me/diary_musician_analyst "Дневник аналитика")

GitHub: [GitHub](https://github.com/Malakhova-Natalya "GitHub")

Habr: [Habr](https://habr.com/ru/users/Malakhova-Natalya/publications/articles/ "Habr")

Спасибо за внимание!